In [ ]:
## before anything you need to visit the site below and make sure you have a google earth engine account
## this is so you can access Sentinel-1 GRD and Sentinel-2 TOA and SR products, as well as other sensor packages and data types

## visit the below website below to setup an earth engine account, enable a cloud project, and enable the ee API 
## https://developers.google.com/earth-engine/cloud/earthengine_cloud_project_setup#get-access-to-earth-engine

In [3]:
import ee
import geemap
import math

In [ ]:
## only need to run this once
## after authenticating with google earth engine you will only need to initialize each session

## https://developers.google.com/earth-engine/guides/auth
ee.Authenticate()

In [4]:
## init ee cloud project you made during initial setup
ee.Initialize(project = 'ee-claycaldgsl') ##enter your project name here as a string to initialize exchanges with ee api

# Functions

In [1]:
## Function to add RGB images to the map.
def add_rgb_to_map(image, map_object):

    date = ee.Date(image.get('date')).format('YYYY-MM-dd').getInfo()
    map_object.addLayer(image, {'min': 0, 'max': 2000, 'bands': ['B4', 'B3', 'B2']}, f'{date}_rgb')

## Function to add spectral indices images to the map.
def add_ind_to_map(image, map_object, band):

    date = ee.Date(image.get('date')).format('YYYY-MM-dd').getInfo()
    if band =='NDWI':
        map_object.addLayer(image, {'min': -1, 'max': 1, 'bands': band, 'palette': cm.palettes.ndwi}, f'{date}_{band}')
    elif band =='NDVI': 
        map_object.addLayer(image, {'min': -1, 'max': 1, 'bands': band, 'palette': cm.palettes.ndvi}, f'{date}_{band}')
    elif band == 'MSAVI2':
        map_object.addLayer(image, {'min': -1, 'max': 1, 'bands': band, 'palette': cm.palettes.RdYlGn}, f'{date}_{band}')
    elif band == 'BSI':
        map_object.addLayer(image, {'min': -1, 'max': 1, 'bands': band, 'palette': cm.palettes.Greens}, f'{date}_{band}')


## functiont to create three important and popular spectral indices
## ndvi = Normalized Difference Vegetation Index, good for vegetation health and cover
## ndwi = Normalized Difference Water Index, good for identifying water bodies and mositure in surface
def s2_10m_target_indices(image):
    # Calculate NDVI
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    # Calculate NDWI
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    # Calculate MSAVI2
    msavi2 = image.expression(
        '((2 * NIR + 1) - ((2 * NIR + 1) ** 2 - 8 * (NIR - RED)) ** 0.5) / 2',
        {
            'NIR': image.select('B8'),
            'RED': image.select('B4')
        }
    ).toFloat().rename('MSAVI2')
    
    # Resample the SWIR1 band to match the resolution of the other bands (10 meters)
    # swir1_resampled = image.select('B11').reproject(crs = msavi2.projection(), scale = 10)
    
    # Calculate the Bare-Soil Index (BSI)
    bsi = image.expression(
        '((SWIR1 + RED) - (NIR + BLUE)) / ((SWIR1 + RED) + (NIR + BLUE))',
        {
            'RED': image.select('B4'),
            'NIR': image.select('B8'),
            'BLUE': image.select('B2'),
            'SWIR1': image.select('B11').resample(mode = 'bicubic') ### scale of 10m specified when exporting images
        }
    ).rename('BSI')
    
    # Add all indices as new bands to the image
    return image.addBands([ndvi, ndwi, msavi2, bsi])

## collects sentinel-1 GRD (radar, no phase) and Sentinel-2 SR (multispectral, adjusted for top of atmosphere reflectance)
def get_sentinel_imagery(aoi, start_date, end_date, s2_cloud_cov, orbit):

    ## Sentinel-2 Surface Reflectance Harmonized ImageCollection
    s2_10m = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
               .filterBounds(aoi)
               .filterDate(ee.Date(start_date), ee.Date(end_date))
               .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', s2_cloud_cov))
               .sort('date')
               .select(['B2', 'B3', 'B4', 'B8', 'B11'])
    )

    ## Clip all images in the collection to the AOI
    s2_10m = s2_10m.map(lambda img: img.clip(aoi))
    ## Apply indices to the Sentinel-2 images
    s2_10m_indices = s2_10m.map(s2_10m_target_indices).select(['NDVI', 'NDWI', 'MSAVI2', 'BSI'])

    # return s1_VV, s1_VH, s2_10m, s2_10m_ndvi, s2_10m_ndwi, s2_10m_msavi2, s2_10m_bsi
    return s2_10m, s2_10m_indices #s2_10m_ndvi, s2_10m_ndwi, s2_10m_msavi2, s2_10m_bsi

## fucntion to get the date of each image in the image collection
def get_date(image):
    return ee.Feature(None, {'date': image.date().format('YYYY-MM-dd')})

def export_image_to_drive(collection, collname, image, index):
    # Define the description for the export, incorporating the index for uniqueness

    if collname == 'rgb':
        description = f's2_10m_{index}'
    elif collname == 'indices':
        description = f's2_10m_{index}'
    elif collname == 'ndvi':
        description = f's2_10m_ndvi_{index}'
    else:
        description = f's2_glcm_{index}'

    # Setup the export task
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        region=aoi,  # Make sure the geometry is defined earlier
        fileFormat='GeoTIFF',
        scale = 10
    )
    task.start()
    print(f'Exporting {description} to Drive...')

def export_all_images(collection, collname):
    image_list = collection.toList(collection.size())  # Convert ImageCollection to List
    num_images = image_list.size().getInfo()  # Get the number of images
    
    for i, date in enumerate(s2_date_list):
        image = ee.Image(image_list.get(i))
        export_image_to_drive(collection, collname, image, date[:10])

def get_s2_glcm(image):

    texture = image.glcmTexture(size=5)


    # Add these metrics as new bands to the original image
    return image.addBands(texture)

# Get area of interest
1. Sabine National Wildlife Refuge (Louisiana) -- Gulf Coast
2. Deer Island (Mississippi) -- Gulf Coast
3. SMIIL (New Jersey) -- East Coast
4. Poplar Island (Maryland) -- East Coast
5. South Bay Salt Pond Restoration (California) -- West Coast
6. Niaqually National Wildlife Refuge (Washington) -- West Coast
7. Kachemak Bay National Estuarine Research Researve (Alaska) -- Arctic 
8. Mendenhall Wetlands (Alaska) -- Arctic

Going to look for sites on the EWN as well

In [7]:
## interactive map for you to draw a polygon to signify your aoi

## Create a map centered at a specific location
m = geemap.Map(center=[20, 0], zoom=2, basemap='HYBRID')
## Add drawing tools
m.add_draw_control()
## Display the map
display(m)

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…

In [8]:
## Get the drawn features
draw_features = m.draw_features[0]
## Establish ee.Polygon from drawn area of interest to collect imagery
aoi = ee.Geometry.Polygon(draw_features.getInfo()['geometry']['coordinates'][0])

In [24]:
aoi

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -74.684715,
              39.108751
            ],
            [
              -74.784622,
              39.152694
            ],
            [
              -74.861526,
              39.041586
            ],
            [
              -74.761963,
              38.996774
            ],
            [
              -74.684715,
              39.108751
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})

# Get Imagery

In [ ]:
start_date = '2020-01-01' ## start date of search window
end_date = '2024-11-14' ## end date of search window
s2_cloud_cov = 1 ## percentage of clouds in sentinel-2 multispectral imagery, less means you see more surface
orbit = 'ASCENDING' ## orbit for imagery

collections = {}
for i in range(int(start_date[:4]), int(end_date[:4])):
    # collections[f's2_10m_{i}_{i+1}'], collections[f's2_10m_ndvi_{i}_{i+1}'], collections[f's2_10m_ndwi_{i}_{i+1}'], collections[f's2_10m_msavi2_{i}_{i+1}'], collections[f's2_10m_bsi_{i}_{i+1}']  = get_sentinel_imagery(aoi, f'{i}{start_date[4:]}', f'{i+1}{end_date[4:]}', s2_cloud_cov, orbit)
    collections[f's2_10m_{i}_{i+1}'], collections[f's2_10m_indices_{i}_{i+1}'] = get_sentinel_imagery(aoi, f'{i}{start_date[4:]}', f'{i+1}{end_date[4:]}', s2_cloud_cov, orbit)

In [ ]:
total_colls = {}
# total_colls['s2_10m'], total_colls['s2_10m_ndvi'], total_colls['s2_10m_ndwi'], total_colls['s2_10m_msavi2'], total_colls['s2_10m_bsi']= get_sentinel_imagery(aoi, start_date, end_date, s2_cloud_cov, orbit)
total_colls['s2_10m'], total_colls['s2_10m_indices']= get_sentinel_imagery(aoi, start_date, end_date, s2_cloud_cov, orbit)

In [11]:
colls = []
for i, coll in enumerate(collections):
    if collections[coll].size().getInfo() != 0: # removes image collections that are empty
        colls.append(coll)

for i, coll in enumerate(colls):
    print(f'{i}: {coll}')

0: s1_2020_2021
1: s2_10m_2020_2021
2: s2_10m_indices_2020_2021
3: s1_2021_2022
4: s2_10m_2021_2022
5: s2_10m_indices_2021_2022
6: s1_2022_2023
7: s2_10m_2022_2023
8: s2_10m_indices_2022_2023
9: s1_2023_2024
10: s2_10m_2023_2024
11: s2_10m_indices_2023_2024


# Map imagery

In [ ]:
####rgb map####

Map = geemap.Map()
Map.centerObject(aoi, 12)

# Visualize each image in the ImageCollection.
s2_images = collections[colls[1]].toList(collections[colls[1]].size())
for i in range(collections[colls[1]].size().getInfo()//10):
    image = ee.Image(s2_images.get(-i))
    add_rgb_to_map(image, Map)

# Display the map.
Map.addLayerControl(position = 'topright')
Map

In [ ]:
Map_ind = geemap.Map()

# Visualize each image in the ImageCollection.
s2ind_images = collections[colls[3]].toList(collections[colls[3]].size())
for i in range(collections[colls[3]].size().getInfo()//2):
    image = ee.Image(s2ind_images.get(i))
    add_ind_to_map(image, Map_ind, collections[colls[3]])

# Display the map.
Map_ind.addLayerControl(position = 'topright')
Map_ind

In [ ]:
Map_ind = geemap.Map()

# Visualize each image in the ImageCollection.
s2ind_images = collections[colls[4]].toList(collections[colls[4]].size())
for i in range(collections[colls[4]].size().getInfo()//2):
    image = ee.Image(s2ind_images.get(i))
    add_ind_to_map(image, Map_ind, collections[colls[4]])

# Display the map.
Map_ind.addLayerControl(position = 'topright')
Map_ind

# Export imagery to google drive

In [ ]:
s2_date_list = total_colls['s2_10m'].map(get_date).aggregate_array('date').getInfo()
export_all_images(total_colls['s2_10m'], 'rgb')